通过对句子的语法结构进行分析可以进一步分析句子的含义
NLTK通过命题逻辑、一阶逻辑等方式分析自然语言的含义

In [1]:
import nltk

对语句含义的理解可以考虑通过语法将其转换为计算机更容易理解的语言
NLTK内置了很多文法进行语言的简单翻译
如下边的示例 根据NLTK中内置的文法 将自然语言转换为SQL语句

In [3]:
from nltk import load_parser
cp = load_parser('grammars/book_grammars/sql0.fcfg')
query = 'What cities are located in China'
trees = list(cp.parse(query.split()))
answer = trees[0].label()['SEM']
answer = [s for s in answer if s]
q = ' '.join(answer)
print(q)

SELECT City FROM city_table WHERE Country="china"


NLTK可以将语言用逻辑语言进行表达，通过命题逻辑推理语言的含义
NLTK用命题逻辑表示语言结构中和特定句子连接词相对应的部分

In [4]:
# NLTK中命题逻辑的逻辑运算符有五种
print(nltk.boolean_ops())

negation       	-
conjunction    	&
disjunction    	|
implication    	->
equivalence    	<->
None


NLTK可以将逻辑表达式转换为Expression对象，通过NLTK的推理模块对Expression进行逻辑证明

In [6]:
from nltk.inference import Prover9

# nltk中的Expression对象能够将逻辑表达式，处理成Expression的子类。
read_expr = nltk.sem.Expression.fromstring
result = read_expr('-(P & Q)')
print(result)

# 逻辑证明通过nltk的推理模块进行。例如通过第三方定理证明程序prover9的接口。
# 并且推理机制的输入必须首先转换为逻辑表达式
# 这里因为Prover9安装困难就不演示了
SnF = read_expr('SnF')
NotFns = read_expr('-FnS')
R = read_expr('SnF -> -FnS')
# prover = Prover9()
# result = prover.prove(NotFns, [SnF, R])
# print(result) True

-(P & Q)


NLTK还可以通过Valuation对象来确定逻辑表达式的真值

In [7]:
# Valuation ：从逻辑的基本符号映射到他们的值。
val = nltk.Valuation([('P', True), ('Q', True), ('R', False)])
dom = set()
g = nltk.Assignment(dom)
# 用val初始化模型m
m = nltk.Model(dom, val)
# 使用evaluate()可确定逻辑表达式的真值情况
result = m.evaluate('(P & Q)', g)
print(result)

True


NLTK可以通过将自然语言表达式转换为一阶逻辑来表达语句的含义
一阶逻辑保留命题逻辑的所有布尔运算，将识别命题中的谓词和参数进行分析
一阶逻辑表达式的语法结构通常是为表达式分配类型：实体类型和公式类型
一阶逻辑也可以通过Prover9进行逻辑证明
以下是代码示例

In [8]:
read_expr = nltk.sem.Expression.fromstring
expr = read_expr('walk(angus)', type_check=True)
print(expr.argument)           # 参数
print(expr.argument.type)      # 实体类型
print(expr.function)           # 谓词
print(expr.function.type)      # 公式类型

# 对一阶逻辑进行逻辑证明
NotFnS = read_expr('-north_of(f, s)')
SnF = read_expr('north_of(s, f)')
R = read_expr('all x. all y. (north_of(x, y) -> -north_of(y, x))')
# prover = nltk.Prover9()
# prover.prove(NotFnS, [SnF, R]) # True

angus
e
walk
<e,?>


In [9]:
v = """
    bertie => b
    olive => o
    cyril => c
    boy => {b}
    girl => {o}
    dog => {c}
    walk => {o, c}
    see => {(b, o), (c, b), (o, c)}
    """
val = nltk.Valuation.fromstring(v)
print(val)

{'bertie': 'b',
 'boy': {('b',)},
 'cyril': 'c',
 'dog': {('c',)},
 'girl': {('o',)},
 'olive': 'o',
 'see': {('o', 'c'), ('c', 'b'), ('b', 'o')},
 'walk': {('c',), ('o',)}}


NLTK支持为独立变量赋值，通过Assignment函数将变量和值绑定，并通过evaluate方式计算原子公式的估值

In [10]:
dom = {'b', 'o', 'c'}
g = nltk.Assignment(dom, [('x', 'o'), ('y', 'c')])
print(g)  
v = """
        bertie => b
        olive => o
        cyril => c
        boy => {b}
        girl => {o}
        dog => {c}
        walk => {o, c}
        see => {(b, o), (c, b), (o, c)}
        """
val = nltk.Valuation.fromstring(v)
# 创建模型
m = nltk.Model(dom, val)
# 计算真值
true_value = m.evaluate('see(olive, y)', g)
print(true_value)  # True

true_value = m.evaluate('see(y,x)', g)
print(true_value)    # False

g[c/y][o/x]
True
False


NLTK中的Expression对象提供了satisfiers方法，返回满足公式的所有个体的合集，可以判断某个集合内的变量中满足公式成立条件的变量有哪些

In [11]:
dom = {'b', 'o', 'c'}
g = nltk.Assignment(dom, [('x', 'o'), ('y', 'c')])
read_expr = nltk.sem.Expression.fromstring
fmla = read_expr('girl(x)|boy(x)')
result = m.satisfiers(fmla, 'x', g)
print(result)   # {'b', 'o'}

# A->B可以等价于-A|B
fmla = read_expr('girl(x)->walk(x)')
result = m.satisfiers(fmla, 'x', g)
print(result)   # {'c', 'o', 'b'}

# 全称量化公式。判断g中是否所有成员都满足
result = m.evaluate('all x.(girl(x) -> walk(x))', g)
print(result)  # True


{'o', 'b'}
{'o', 'b', 'c'}
True


基于上述对象和方法，NLTK就可以通过一阶逻辑对句子进行语义分析，如判断某个句子是否有歧义

In [12]:
v = """
    bruce => b
    elspeth => e
    julia => j
    matthew => m
    person => {b, e, j, m}
    admire => {(j, b), (b, b), (m, e), (e, m)}
    """
val = nltk.Valuation.fromstring(v)

read_expr = nltk.sem.Expression.fromstring
dom = val.domain
m = nltk.Model(dom, val)
g = nltk.Assignment(dom)
fmla = read_expr('(person(x) -> exists y.(person(y) & admire(x, y)))')
result = m.satisfiers(fmla, 'x', g)
print(result)   # {'e', 'j', 'm', 'b'}

fmla = read_expr('(person(y) & all x.(person(x) -> admire(x, y)))')
result = m.satisfiers(fmla,'y',g)
print(result) 

{'m', 'e', 'b', 'j'}
set()
